In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

# USD/KRW 환율 데이터 다운로드
usd_krw = yf.Ticker("KRW=X")
data = usd_krw.history(period="5y")

# 종가 데이터만 사용
df = data['Close'].values.reshape(-1, 1)

# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

# 훈련 데이터 생성
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 100
X, y = create_dataset(df_scaled, time_step)

# 데이터 분할 (훈련:테스트 = 80:20)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 데이터 reshape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(100, 1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# 모델 훈련
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1)

# 예측
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# 예측값 역정규화
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# 결과 시각화
plt.figure(figsize=(16,8))
plt.plot(scaler.inverse_transform(df_scaled))
plt.plot(range(100, len(train_predict) + 100), train_predict, color='red')
plt.plot(range(len(train_predict) + 100, len(train_predict) + len(test_predict) + 100), test_predict, color='green')
plt.legend(['실제 데이터', '훈련 데이터 예측', '테스트 데이터 예측'])
plt.title('USD/KRW 환율 예측')
plt.xlabel('날짜')
plt.ylabel('환율')
plt.show()

# 다음 30일 예측
last_100_days = df_scaled[-100:]
X_future = []
for i in range(30):
    if i == 0:
        X_future.append(last_100_days)
    else:
        X_future.append(np.vstack((X_future[i-1][1:], [[X_future[i-1][-1, 0]]])))

X_future = np.array(X_future)
future_predict = model.predict(X_future)
future_predict = scaler.inverse_transform(future_predict)

print("다음 30일 예측 환율:")
for i, price in enumerate(future_predict):
    print(f"{i+1}일 후: {price[0]:.2f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
 6/16 ━━━━━━━━━━━━━━━━━━━━ 3s 324ms/step - loss: 0.1555

KeyboardInterrupt: 